In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy import unique
from keras.models import Sequential
from keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split


In [16]:
# Importing the data
df_answers = pd.read_csv(r'C:\Users\casbacar\Documents\Study\Specialization\Supervised marchine learning\Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv')
df_unscaled = pd.read_csv(r'C:\Users\casbacar\Documents\Study\Specialization\Data Sets\Dataset-weather-prediction-dataset-processed.csv')

In [17]:
# For our analysis we'll proceed with an RNN model because its better at handling tempoeral data and predicting future events based on past data

In [18]:
# For the pleasant weather dataset, we drop the DATE column 
answers = df_answers.drop(['DATE'],axis=1)

In [19]:
df_unscaled.shape

(22950, 170)

In [44]:
# We need to remove some observations missing data

unscaled_c = df_unscaled.drop(columns=['GDANSK_cloud_cover',
                                      'GDANSK_humidity',
                                      'GDANSK_precipitation',
                                      'GDANSK_snow_depth',
                                      'GDANSK_temp_mean',
                                      'GDANSK_temp_min',
                                      'GDANSK_temp_max',
                                      'ROMA_cloud_cover',
                                      'ROMA_wind_speed',
                                      'ROMA_humidity',
                                      'ROMA_pressure',
                                      'ROMA_sunshine',
                                      'ROMA_temp_mean',
                                      'TOURS_wind_speed',
                                      'TOURS_humidity',
                                      'TOURS_pressure',
                                      'TOURS_global_radiation',
                                      'TOURS_precipitation',
                                      'TOURS_temp_mean',
                                      'TOURS_temp_min',
                                      'TOURS_temp_max',
                                      'BASEL_precipitation',
                                      'BASEL_snow_depth',
                                      'DUSSELDORF_precipitation',
                                      'DUSSELDORF_snow_depth',
                                      'HEATHROW_precipitation',
                                      'HEATHROW_snow_depth',
                                      'MUNCHENB_precipitation',
                                      'MUNCHENB_snow_depth',
                                      'OSLO_precipitation',
                                      'OSLO_snow_depth',
                                      'VALENTIA_precipitation',
                                      'VALENTIA_snow_depth','STOCKHOLM_sunshine','STOCKHOLM_precipitation'])


In [21]:
weather = {
    0: 'sunny',
    1: 'rainy',
    2: 'cloudy',
    3: 'snowy'
}

In [22]:
weather_labels = [
    'BASEL_pleasant_weather', 'BELGRADE_pleasant_weather',
    'BUDAPEST_pleasant_weather', 'DEBILT_pleasant_weather',
    'DUSSELDORF_pleasant_weather', 'HEATHROW_pleasant_weather',
    'KASSEL_pleasant_weather', 'LJUBLJANA_pleasant_weather',
    'MAASTRICHT_pleasant_weather', 'MADRID_pleasant_weather',
    'MUNCHENB_pleasant_weather', 'OSLO_pleasant_weather',
    'SONNBLICK_pleasant_weather', 'STOCKHOLM_pleasant_weather',
    'VALENTIA_pleasant_weather'
]

# Create the dictionary
weather = {i: label for i, label in enumerate(weather_labels)}

In [45]:
unscaled_c.shape

(22950, 135)

In [24]:
answers.shape

(22950, 15)

In [36]:
unscaled_c.head()

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,2.1,0.85,1.018,0.32,0.7,6.5,0.8,10.9,1,...,2.2,4.9,5,0.88,1.0003,0.45,4.7,8.5,6.0,10.9
1,6,2.1,0.84,1.018,0.36,1.1,6.1,3.3,10.1,6,...,3.0,5.0,7,0.91,1.0007,0.25,0.7,8.9,5.6,12.1
2,8,2.1,0.90,1.018,0.18,0.0,8.5,5.1,9.9,6,...,1.3,4.1,7,0.91,1.0096,0.17,0.1,10.5,8.1,12.9
3,3,2.1,0.92,1.018,0.58,4.1,6.3,3.8,10.6,8,...,0.4,2.3,7,0.86,1.0184,0.13,0.0,7.4,7.3,10.6
4,6,2.1,0.95,1.018,0.65,5.4,3.0,-0.7,6.0,8,...,0.8,4.3,3,0.80,1.0328,0.46,5.7,5.7,3.0,8.4


In [37]:
answers.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
# Exporting the datasets as they'll be needed in a future exercise
unscaled_c.to_csv('C:\\Users\\casbacar\\Documents\\Study\\Specialization\\Data sets\\unscaled_exercise_clean_2.2.csv')
answers.to_csv('C:\\Users\\casbacar\\Documents\\Study\\Specialization\\Data sets\\pleasant_weather_answers_clean_2.2.csv')

In [26]:
X = unscaled_c
y = answers

(22950, 133)

In [63]:
X = np.reshape(X, (-1, 15, 9))

In [64]:
X.shape

(22950, 15, 9)

In [68]:
#Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [69]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(18360, 15, 9) (18360, 15)
(4590, 15, 9) (4590, 15)


In [70]:
len(X_train[0])

15

In [71]:
len(X_train[0][0])

9

In [74]:
y_train.values[0]

array([1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0], dtype=int64)

In [203]:
epochs = 30
batch_size = 30
n_hidden = 128

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train.values[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid')) #Don't use relu here!

C:\Users\casbacar\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [204]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [205]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/30
612/612 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.2465 - loss: 11.7693 - val_accuracy: 0.2673 - val_loss: 9.9620
Epoch 2/30
612/612 ━━━━━━━━━━━━━━━━━━━━ 20s 19ms/step - accuracy: 0.4717 - loss: 12.0622 - val_accuracy: 0.6447 - val_loss: 10.7201
Epoch 3/30
612/612 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - accuracy: 0.6423 - loss: 12.6827 - val_accuracy: 0.6468 - val_loss: 11.3511
Epoch 4/30
612/612 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.6423 - loss: 13.2618 - val_accuracy: 0.6479 - val_loss: 12.3383
Epoch 5/30
612/612 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.6419 - loss: 13.8845 - val_accuracy: 0.6479 - val_loss: 13.0813
Epoch 6/30
612/612 ━━━━━━━━━━━━━━━━━━━━ 22s 19ms/step - accuracy: 0.6423 - loss: 14.4477 - val_accuracy: 0.6486 - val_loss: 13.4236
Epoch 7/30
612/612 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.6423 - loss: 14.9414 - val_accuracy: 0.6479 - val_loss: 14.1570
Epoch 8/30
612/612 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.6425 - lo

In [206]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([weather_labels[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([weather_labels[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [207]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
Pred                         BASEL_pleasant_weather  MADRID_pleasant_weather
True                                                                        
BASEL_pleasant_weather                         2974                        0
BELGRADE_pleasant_weather                       862                        0
BUDAPEST_pleasant_weather                       161                        0
DEBILT_pleasant_weather                          69                        0
DUSSELDORF_pleasant_weather                      26                        0
HEATHROW_pleasant_weather                        77                        0
KASSEL_pleasant_weather                          11                        0
LJUBLJANA_pleasant_weather                       47                        0
MAASTRICHT_pleasant_weather                       3                        0
MADRID_pleasant_weather                         342                        1
MUNCHENB_pleasant_weather         